chicken dinner github : https://github.com/crflynn/chicken-dinner
<br>pubg api-assets : https://github.com/pubg/api-assets

### Aggregate statistics

import library

In [1]:
## for calling dataset
import requests

## parsing dataset
from chicken_dinner.pubgapi import PUBG
from chicken_dinner.constants import COLORS
from chicken_dinner.constants import map_dimensions

import json
import numpy as np
import pandas as pd

from tqdm import tqdm
from datetime import datetime
import config

In [2]:
api_key = config.api_key
match_sample_url = config.match_sample_url

requesting PUBG API data

In [3]:
today = datetime.today().strftime('%Y%m%d_%H%M')
today

'20240210_2233'

In [4]:
headers = {
    'accept': 'application/vnd.api+json',
    'Authorization': f'Bearer {api_key}',
}

In [5]:
response = requests.get(match_sample_url, headers=headers)
response

<Response [200]>

In [6]:
r_dict = json.loads(response.text)

In [7]:
## save data into json file
sample_file_path = f'data/{today}_smp_data.json'

In [8]:
with open(sample_file_path, 'w') as f:
    json.dump(r_dict, f)

reloading the previously saved sample data

In [9]:
with open(sample_file_path, 'r') as f:
    sample_data = json.load(f)

In [10]:
sample_data

{'data': {'type': 'sample',
  'id': '9d0e3a2a-eb43-4c9d-8683-4ec00daf3256',
  'attributes': {'createdAt': '2024-02-09T00:00:00Z',
   'titleId': 'bluehole-pubg',
   'shardId': 'kakao'},
  'relationships': {'matches': {'data': [{'type': 'match',
      'id': '8cc6c687-657b-48a0-8282-9e6b7e16d2ce'},
     {'type': 'match', 'id': 'ef645db7-a848-416c-89de-85f272b993df'},
     {'type': 'match', 'id': 'd892a132-a581-40a6-8354-290f808a3595'},
     {'type': 'match', 'id': '092188b5-ddf7-4c87-9b99-e929c36e9769'},
     {'type': 'match', 'id': 'c37c341f-d968-4374-804c-7c2cdd3a62a6'},
     {'type': 'match', 'id': '875885e7-aa97-42ce-a6a1-b8c14afb78ca'},
     {'type': 'match', 'id': 'b507d7ee-e9ca-46cc-88d0-ea81ce8741a2'},
     {'type': 'match', 'id': '43b94883-fa77-4770-baaf-7d88723ae59d'},
     {'type': 'match', 'id': '580747c9-597c-4394-bf23-e0e6408e0264'},
     {'type': 'match', 'id': '94be7c3c-a54b-4d36-aec9-3943f00ddc1a'},
     {'type': 'match', 'id': 'f5fedc31-3d0c-4f0b-b215-7f32bf0fa603'},
   

In [11]:
match_list = sample_data['data']['relationships']['matches']['data']
len(match_list), match_list[:5]

(171,
 [{'type': 'match', 'id': '8cc6c687-657b-48a0-8282-9e6b7e16d2ce'},
  {'type': 'match', 'id': 'ef645db7-a848-416c-89de-85f272b993df'},
  {'type': 'match', 'id': 'd892a132-a581-40a6-8354-290f808a3595'},
  {'type': 'match', 'id': '092188b5-ddf7-4c87-9b99-e929c36e9769'},
  {'type': 'match', 'id': 'c37c341f-d968-4374-804c-7c2cdd3a62a6'}])

Work on each match to get more information
- first save it into df
- then save the dataset into json file

In [12]:
features = ['date', 'match_id', 'duration', 'game_mode', 
            'num_players', 'num_teams', 'map']

In [13]:
agg_df = pd.DataFrame(columns = features)

In [14]:
agg_df

,date,match_id,duration,game_mode,num_players,num_teams,map


In [15]:
pubg = PUBG(api_key, shard = 'kakao')

In [16]:
error_list = []
for i, match in tqdm(enumerate(match_list)):
    try:
        match_id = match['id']
        current_match = pubg.match(match_id)
        telemetry = current_match.get_telemetry()
    
        date = telemetry.started()
        duration = telemetry.match_length()
        game_mode = current_match.game_mode
        num_players = telemetry.num_players()
        num_teams = telemetry.num_teams()
        map = telemetry.map_name()
    
        agg_df.loc[len(agg_df)] = [date, match_id, duration, 
                                   game_mode, num_players, num_teams, map]
    except:
        error_list.append(match_id)
        print(match['id'])
        continue

107it [11:32,  6.47s/it]


In [17]:
# for match in error_list:
#     current_match = pubg.match(match)
#     telemetry = current_match.get_telemetry()
    
#     date = telemetry.started()
#     duration = telemetry.match_length()
#     game_mode = current_match.game_mode
#     num_players = telemetry.num_players()
#     num_teams = telemetry.num_teams()
#     map = telemetry.map_name()
    
#     agg_df.loc[len(agg_df)] = [date, match_id, duration, 
#                                game_mode, num_players, num_teams, map]

In [18]:
agg_df.shape, agg_df.head()

((107, 7),
                            date                              match_id  \
 0  2024-02-05T23:03:43.9531715Z  858cc73c-5926-497d-a975-a55b49975a12   
 1   2024-02-05T23:54:19.261471Z  e2b8a957-c996-4d4b-af91-4d67d803a71c   
 2  2024-02-06T00:36:11.4290998Z  7a53a8aa-b9a1-4af4-82e7-2fa4f516d671   
 3  2024-02-06T01:21:15.1780711Z  ff9530d7-208b-46d5-a244-cc2f52bf258f   
 4  2024-02-06T01:20:36.3554113Z  514db3d4-8547-414d-9f9e-04472b7a1b5f   
 
   duration game_mode  num_players  num_teams                   map  
 0      455  clansolo            1          0           Camp Jackal  
 1     1549      solo          100        100  Erangel (Remastered)  
 2     1384     squad          100         27                Sanhok  
 3     1264      solo          100        100                Sanhok  
 4     1334       duo          100         51  Erangel (Remastered)  )

#### save the aggregate data

In [19]:
json_data = agg_df.to_json(orient='records')
# len(json_data), json_data

In [20]:
json_file_name = f'data/{today}_agg_data.json'

In [21]:
with open(json_file_name, 'w') as f:
    json.dump(json_data, f)

#### check existing file

In [22]:
import glob

In [23]:
check_file = glob.glob('data/*_agg_data.json')
check_file

['data/20240207_2335_agg_data.json',
 'data/20240204_2224_agg_data.json',
 'data/20240206_2302_agg_data.json',
 'data/20240202_2128_agg_data.json',
 'data/20240205_2308_agg_data.json']

In [24]:
with open(check_file[0], 'r') as f:
    check_data = json.load(f)

In [25]:
check_data = json.loads(check_data)
check_data

[{'date': '2024-02-05T23:03:43.9531715Z',
  'match_id': '858cc73c-5926-497d-a975-a55b49975a12',
  'duration': 455,
  'game_mode': 'clansolo',
  'num_players': 1,
  'num_teams': 0,
  'map': 'Camp Jackal'},
 {'date': '2024-02-05T23:54:19.261471Z',
  'match_id': 'e2b8a957-c996-4d4b-af91-4d67d803a71c',
  'duration': 1549,
  'game_mode': 'solo',
  'num_players': 100,
  'num_teams': 100,
  'map': 'Erangel (Remastered)'},
 {'date': '2024-02-06T00:36:11.4290998Z',
  'match_id': '7a53a8aa-b9a1-4af4-82e7-2fa4f516d671',
  'duration': 1384,
  'game_mode': 'squad',
  'num_players': 100,
  'num_teams': 27,
  'map': 'Sanhok'},
 {'date': '2024-02-06T01:21:15.1780711Z',
  'match_id': 'ff9530d7-208b-46d5-a244-cc2f52bf258f',
  'duration': 1264,
  'game_mode': 'solo',
  'num_players': 100,
  'num_teams': 100,
  'map': 'Sanhok'},
 {'date': '2024-02-06T01:20:36.3554113Z',
  'match_id': '514db3d4-8547-414d-9f9e-04472b7a1b5f',
  'duration': 1334,
  'game_mode': 'duo',
  'num_players': 100,
  'num_teams': 51,


In [ ]:
# check_data = pd.DataFrame(check_data)
check_data.head()

In [ ]:
check_data.info()